In [1]:
from pprint import pprint
# Import statements
import numpy as np
import plotly
import plotly.graph_objs as go
import pandas as pd
from dotenv import dotenv_values
import plotly.graph_objects as go
SECRETS = dotenv_values("../envs/.env")

TRAIN_DATA_PATH = SECRETS['TRAIN_DATA_PATH']
TEST_DATA_PATH = SECRETS['TEST_DATA_PATH']
PULL_UP_STRENGTH, PULL_DOWN_STRENGTH = list(map(int, SECRETS['BETA'].split(':')))
NUM_SAMPLES = float(SECRETS['NUM_SAMPLES'])

In [2]:
from simulation.Simulation import Simulation
from circuit.circuit import Circuit
from circuit.gate import Gate
from circuit.eos import EndOfSequence
from circuit.voltagesource import VoltageSource


In [3]:
# class VoltageSource:
#     def __init__(self, config:dict):
#         self.name = config['name']
#         if self._validate_config(config):
#             self.input_config = config
        
#         self.output_node_name = config['name']
#         if not config['ideal']:
#             gate_config = {"name":self.name+"driver", "type":"1NOT", "k":config['k'], "input_components":[self]}
#             self.driver = Gate(gate_config)
#             self.output_node_name = self.driver.output_node_name
        
#         self.netlist = self.__generate_netlist()
    
#     def __generate_netlist(self):
#         if self.output_node_name == self.name:
#             return f"{self.name.upper()} {self.name} 0 pwl(0 0.7 0.9999us 0.7 1us 0 2us 0)\n\n" 
#         else:
#             return f"{self.name.upper()} {self.name} 0 pwl(0 0.7 0.9999us 0.7 1us 0 2us 0)\n" + self.driver.netlist
#     def __repr__(self):
#         return self.__generate_netlist()
    
#     def _validate_config(self, config:dict):
#         if 'name' not in config:
#             raise ValueError("Name of the voltage source not provided")
#         if 'ideal' not in config:
#             raise ValueError("Ideal property of the voltage source not provided")
#         if not config['ideal']:
#             if 'k' not in config:
#                 raise ValueError("Sizing of the driver not provided")
#         if 'ideal' in config and not isinstance(config['ideal'], bool):
#             raise ValueError("Ideal property must be a boolean")
#         if 'k' in config and not isinstance(config['k'], int):
#             raise ValueError("Driver sizing must be an integer")
#         if 'name' in config and not isinstance(config['name'], str):
#             raise ValueError("Name of the voltage source must be a string")
        
#         return True

# class Gate:    
#     VALID_TYPES = {"2AND", "2OR", "1NOT", "2NAND", "2NOR"}
#     def __init__(self, config:dict = {}):
#         self._validate_config(config)
#         if config['type'] not in self.VALID_TYPES:
#             raise ValueError(f"Invalid gate type: {config['type']}. Valid types are: {', '.join(self.VALID_TYPES)}")
#         if int(config['type'][0]) != len(config['input_components']):
#             raise ValueError(f"Number of input components does not match gate type. {config['type']}, {len(config['input_components'])}")
#         self.type = config['type']
#         self.name = config['name']
#         self.output_node_name = f"out_{config['name']}"
#         self.k = config['k']
#         self.input_gates = config['input_components']
#         self.output_nodes = []
#         self.netlist = self.__generate_netlist()


#     def __generate_netlist(self):
#         if self.type == "2NAND":
#             return f"M{self.name}m0 {self.output_node_name} {self.input_gates[0].output_node_name} vdd vdd pmos_lvt nfin = {int(self.k * 3)}\n"+\
#             f"M{self.name}m1 {self.output_node_name} {self.input_gates[1].output_node_name} vdd vdd pmos_lvt nfin = {int(self.k * 3)}\n"+\
#             f"M{self.name}m2 {self.output_node_name} {self.input_gates[0].output_node_name} {self.name}mid 0 nmos_lvt nfin = {int(self.k*4)}\n"+\
#             f"M{self.name}m3 {self.name}mid {self.input_gates[1].output_node_name} 0 0 nmos_lvt nfin = {int(self.k*4)}\n\n"
            
#         elif self.type == "1NOT":
#             return f"M{self.name}m0 {self.output_node_name} {self.input_gates[0].output_node_name} vdd vdd pmos_lvt nfin = {int(self.k * 3)}\n"+\
#             f"M{self.name}m1 {self.output_node_name} {self.input_gates[0].output_node_name} 0 0 nmos_lvt nfin = {int(self.k*2)}\n\n"
        
#         elif self.type == "2NOR":
#             return f"M{self.name}m0 {self.output_node_name} {self.input_gates[0].output_node_name} {self.name}mid vdd pmos_lvt nfin = {int(self.k*6)}\n"+\
#                 f"M{self.name}m1 {self.name}mid {self.input_gates[1].output_node_name} vdd vdd pmos_lvt nfin = {int(self.k*6)}\n"+\
#                 f"M{self.name}m2 {self.output_node_name} {self.input_gates[0].output_node_name} 0 0 nmos_lvt nfin = {int(self.k*2)}\n"+\
#                 f"M{self.name}m3 {self.output_node_name} {self.input_gates[1].output_node_name} 0 0 nmos_lvt nfin = {int(self.k*2)}\n\n"


#     def __repr__(self):
#         return self.__generate_netlist()
    
#     def _validate_config(self, config:dict):
#         if 'type' not in config:
#             raise ValueError("Type of the gate not provided")
#         if 'input_components' not in config:
#             raise ValueError("Input components of the gate not provided")
#         if 'name' not in config:
#             raise ValueError("Name of the gate not provided")
#         if 'k' not in config:
#             raise ValueError("Sizing of the gate not provided")
#         if 'type' in config and config['type'] not in self.VALID_TYPES:
#             raise ValueError(f"Invalid gate type: {config['type']}. Valid types are: {', '.join(self.VALID_TYPES)}")
#         if 'input_components' in config and not isinstance(config['input_components'], list):
#             raise ValueError("Input components of the gate must be a list")
#         if 'name' in config and not isinstance(config['name'], str):
#             raise ValueError("Name of the gate must be a string")
#         if 'k' in config and not isinstance(config['k'], int):
#             raise ValueError("Sizing of the gate must be an integer")
#         return True
    

# class EndOfSequence:
#     def __init__(self, config:dict = {}):
#         self._validate_config(config)
#         self.k = config['k']
#         self.name = config['name']
#         self.input_gate = config['input_gate']
#         self.capacitance = config['capacitance']
#         gate_config = {"name":f"{self.name}EOS", "type":"1NOT", "k":config['k'], "input_components":[config['input_gate']]}
#         self.eos = Gate(gate_config)
#         self.netlist = self.__generate_netlist()
#     def __generate_netlist(self):
#         return self.eos.netlist + f"C{self.name}eos {self.eos.output_node_name} 0 {self.capacitance}f\n"
#     def __repr__(self):
#         return self.__generate_netlist()
    
#     def _validate_config(self, config):
#         required_keys = {
#             'k': int,
#             'input_gate': Gate,
#             'capacitance': (float, int),
#             'name': str
#         }
        
#         for key, expected_type in required_keys.items():
#             if key not in config:
#                 raise ValueError(f"{key.replace('_', ' ').capitalize()} not provided")
#             if not isinstance(config[key], expected_type):
#                 if isinstance(expected_type, tuple):
#                     type_names = ' or '.join(t.__name__ for t in expected_type)
#                     raise ValueError(f"{key.replace('_', ' ').capitalize()} must be {type_names}")
#                 else:
#                     raise ValueError(f"{key.replace('_', ' ').capitalize()} must be {expected_type.__name__}")
        
#         return True

In [4]:
# """
# The circuit class gets a name, a dictionary of gate names and their corresponding gate objects,
# a dictionary of voltage source names , and an end of sequence capacitance and sizing.
# The user will provide a topologically sorted dictionary of gate names and their corresponding gate objects.
# structure of the gates dictionary:
# {
#     "gate_name": {"type": str, "k": int, "input_components": [input_gate_names or input voltage_source_names : str]}
# }
# structure of the voltage sources dictionary:
# {
#     "voltage_source_name": {"ideal": bool, "k": int}
# }
# Structure of the eos_dict dictionary:
# {
#     "k": int,
#     "input_gate": "gate_name",
#     "capacitance": float
# }
# inverting: bool


# The circuit class will generate a netlist for the circuit and write it to a file.
# The circuit class will also generate a spice simulation for the circuit and write it to a file.
# example:
# circuit1 = Circuit("circuit1", 
#                     {
#                         "gate1": {"type":"2NOR", "k":2, "input_components":["v1", "v2"]},
#                         "gate2": {"type":"1NOT", "k":10, "input_components":["gate1"]}
#                     },
#                     {
#                         "v1": { "ideal": False, "k": 1},
#                         "v2": { "ideal": False, "k": 1}
#                     },
#                     {
#                         "k": 1,
#                         "input_gate": "gate1",
#                         "capacitance": 9
#                     }, 
#                     inverting=True
#                     )

# """


# class Circuit:
#     def __init__(self, name:str, gate_dict:dict, voltage_dict:dict, eos_dict:dict, inverting = False):
#         self.name = name
#         self.gates = {}
#         self.voltage_sources = {}
#         self.eos = None
#         self.netlist = ""
#         voltage_dict = {key + name: value for key, value in voltage_dict.items()}
#         gate_dict = {key + name: value for key, value in gate_dict.items()}
#         self.inverting = inverting

#         for _, value in gate_dict.items():
#             value['input_components'] = [item + name for item in value['input_components']]
        
#         eos_dict["input_gate"] = eos_dict["input_gate"] + name
        
#         self.__generate_voltage_sources(voltage_dict)
#         self.__generate_gates(gate_dict)
#         self.__generate_eos(eos_dict)
#         self.__generate_netlist()
    
#     def __generate_voltage_sources(self, voltage_dict):
#         for name, params in voltage_dict.items():
#             params['name'] = name
#             self.voltage_sources[name] = VoltageSource(params)
    
#     def __generate_gates(self, gate_dict):
#         for name, params in gate_dict.items():
#             input_components = []
#             for input_name in params['input_components']:
#                 if input_name in self.gates:
#                     input_components.append(self.gates[input_name])
#                 elif input_name in self.voltage_sources:
#                     input_components.append(self.voltage_sources[input_name])
#             gate_config = {"name":name, "type":params['type'], "k":params['k'], "input_components":input_components}
#             self.gates[name] = Gate(gate_config)
    
#     def __generate_eos(self, eos_dict):
#         eos_dict["name"] = self.name
#         eos_dict["input_gate"] = self.gates[eos_dict["input_gate"]]
#         self.eos = EndOfSequence(eos_dict)

#     def return_netlist(self):
#         netlist = ""
#         for voltage_source in self.voltage_sources.values():
#             netlist += voltage_source.netlist
#         for gate in self.gates.values():
#             netlist += gate.netlist
#         netlist += str(self.eos)

#         if self.inverting:
#             simulation_statement = f".MEASURE TRAN tdlay{self.name} TRIG V({self.voltage_sources[list(self.voltage_sources.keys())[0]].output_node_name})"+\
#         f" VAL = 0.35 TD = 0n RISE = 1 TARG V({self.eos.input_gate.output_node_name}) VAL = 0.35  FALL = 1\n"
#         else:
#             simulation_statement = f".MEASURE TRAN tdlay{self.name} TRIG V({self.voltage_sources[list(self.voltage_sources.keys())[0]].output_node_name})"+\
#         f" VAL = 0.35 TD = 0n RISE = 1 TARG V({self.eos.input_gate.output_node_name}) VAL = 0.35  RISE = 1\n"

        
#         return netlist, simulation_statement


#     def __generate_netlist(self):
        
#         self.netlist = f"/////////////////////3 STAGE NAND/////////////////////////////\n"+\
#                 ".inc \"/home/lalithsai20/EMDproject/7nm_TT_160803.pm\"\n\n"
#         self.netlist += self.return_netlist()[0]
#         self.netlist += "\nVdd vdd 0 0.7\n"
#         self.netlist += f"\n.option post\n"+\
#                         ".tran 1p 2u\n"
#         self.netlist += self.return_netlist()[1]
#         self.netlist += ".end\n"
#         return self.netlist
    
#     def __repr__(self):
#         return self.__generate_netlist()

#     def save_circuit_to_file(self, file_path:str):
#         with open(file_path, "w") as file:
#             file.write(self.netlist)




In [5]:
# class Simulation:
#     def __init__(self, name:str, circuits:list):
#         self.name = name
#         self.netlist = self.__generate_netlist(circuits)
    
#     def __generate_netlist(self, circuits):
#         self.netlist = f"/////////////////////3 STAGE NAND/////////////////////////////\n"+\
#                 ".inc \"/home/lalithsai20/EMDproject/7nm_TT_160803.pm\"\n\n"
#         for circuit in circuits:
#             self.netlist += circuit.return_netlist()[0]
#         self.netlist += "\nVdd vdd 0 0.7\n"
#         self.netlist += f"\n.option post\n"+\
#                         ".tran 1p 2u\n"
#         for circuit in circuits:
#             self.netlist += circuit.return_netlist()[1]
#         self.netlist += ".end\n"

#         return self.netlist
    
#     def __repr__(self):
#         return self.netlist
    
#     def save(self, file_path:str):
#         with open(file_path, "w") as file:
#             file.write(self.netlist)

In [6]:
circuit1 = Circuit("circuit1", 
                    {
                        "gate1": {"type":"2NOR", "k":2, "input_components":["v1", "v2"]}
                    },
                    {
                        "v1": { "ideal": False, "k": 1},
                        "v2": { "ideal": False, "k": 1}
                    },
                    {
                        "k": 1,
                        "input_gate": "gate1",
                        "capacitance": 9
                    }, inverting=True)
circuit2 = Circuit("circuit2",
                    {
                        "gate1": {"type":"2NOR", "k":2, "input_components":["v1", "v2"]},
                        "gate2": {"type":"1NOT", "k":9, "input_components":["gate1"]},
                        "gate3": {"type":"1NOT", "k":18, "input_components":["gate2"]}
                    },
                    {
                        "v1": { "ideal": False, "k": 8},
                        "v2": { "ideal": False, "k": 8}
                    },
                    {
                        "k": 13,
                        "input_gate": "gate3",
                        "capacitance": 20
                    })

simulation = Simulation("simulation", [circuit1, circuit2])

In [7]:
circuit2

/////////////////////3 STAGE NAND/////////////////////////////
.inc "/home/lalithsai20/EMDproject/7nm_TT_160803.pm"

V1CIRCUIT2 v1circuit2 0 pwl(0 0.7 0.9999us 0.7 1us 0 2us 0)
Mv1circuit2driverm0 out_v1circuit2driver v1circuit2 vdd vdd pmos_lvt nfin = 24
Mv1circuit2driverm1 out_v1circuit2driver v1circuit2 0 0 nmos_lvt nfin = 16

V2CIRCUIT2 v2circuit2 0 pwl(0 0.7 0.9999us 0.7 1us 0 2us 0)
Mv2circuit2driverm0 out_v2circuit2driver v2circuit2 vdd vdd pmos_lvt nfin = 24
Mv2circuit2driverm1 out_v2circuit2driver v2circuit2 0 0 nmos_lvt nfin = 16

Mgate1circuit2m0 out_gate1circuit2 out_v1circuit2driver gate1circuit2mid vdd pmos_lvt nfin = 12
Mgate1circuit2m1 gate1circuit2mid out_v2circuit2driver vdd vdd pmos_lvt nfin = 12
Mgate1circuit2m2 out_gate1circuit2 out_v1circuit2driver 0 0 nmos_lvt nfin = 4
Mgate1circuit2m3 out_gate1circuit2 out_v2circuit2driver 0 0 nmos_lvt nfin = 4

Mgate2circuit2m0 out_gate2circuit2 out_gate1circuit2 vdd vdd pmos_lvt nfin = 27
Mgate2circuit2m1 out_gate2circuit2 out

In [8]:

simulation = Simulation("simulation", [circuit1, circuit2])
simulation.save(TEST_DATA_PATH + "/simulation1.sp")

In [9]:
circuit1.save_circuit_to_file(TEST_DATA_PATH+"\circuit2.sp")

In [10]:
simulation

/////////////////////3 STAGE NAND/////////////////////////////
.inc "/home/lalithsai20/EMDproject/7nm_TT_160803.pm"

V1CIRCUIT1 v1circuit1 0 pwl(0 0.7 0.9999us 0.7 1us 0 2us 0)
Mv1circuit1driverm0 out_v1circuit1driver v1circuit1 vdd vdd pmos_lvt nfin = 3
Mv1circuit1driverm1 out_v1circuit1driver v1circuit1 0 0 nmos_lvt nfin = 2

V2CIRCUIT1 v2circuit1 0 pwl(0 0.7 0.9999us 0.7 1us 0 2us 0)
Mv2circuit1driverm0 out_v2circuit1driver v2circuit1 vdd vdd pmos_lvt nfin = 3
Mv2circuit1driverm1 out_v2circuit1driver v2circuit1 0 0 nmos_lvt nfin = 2

Mgate1circuit1m0 out_gate1circuit1 out_v1circuit1driver gate1circuit1mid vdd pmos_lvt nfin = 12
Mgate1circuit1m1 gate1circuit1mid out_v2circuit1driver vdd vdd pmos_lvt nfin = 12
Mgate1circuit1m2 out_gate1circuit1 out_v1circuit1driver 0 0 nmos_lvt nfin = 4
Mgate1circuit1m3 out_gate1circuit1 out_v2circuit1driver 0 0 nmos_lvt nfin = 4

Mcircuit1EOSm0 out_circuit1EOS out_gate1circuit1 vdd vdd pmos_lvt nfin = 3
Mcircuit1EOSm1 out_circuit1EOS out_gate1circuit

In [11]:
circuit3 = Circuit("circuit3",
                    {
                        "gate1": ["2NOR", 2, ["v1", "v2"]],
                        "gate2": ["1NOT", 9, ["gate1"]],
                        "gate3": ["2NAND", 18, ["gate2", "gate2"]]
                    },
                    {
                        "v1": [False, 8],
                        "v2": [False, 8]
                    },
                    {
                        "k": 13,
                        "input_gate": "gate3",
                        "capacitance": 20
                    })

circuit4 = Circuit("circuit4",
                    {
                        "gate1": ["2NOR", 2, ["v1", "v2"]],
                        "gate2": ["1NOT", 9, ["gate1"]],
                        "gate3": ["2NAND", 18, ["gate2", "gate2"]],
                        "gate4": ["1NOT", 7, ["gate3"]]
                    },
                    {
                        "v1": [False, 8],
                        "v2": [False, 8]
                    },
                    {
                        "k": 13,
                        "input_gate": "gate3",
                        "capacitance": 20
                    })

sim2 = Simulation("simulation2", [circuit3, circuit4])

TypeError: list indices must be integers or slices, not str

In [ ]:
sim2.save(TEST_DATA_PATH + "/simulation2.sp")

In [ ]:
circuit3

/////////////////////3 STAGE NAND/////////////////////////////
.inc "/home/lalithsai20/EMDproject/7nm_TT_160803.pm"

V1CIRCUIT3 v1circuit3 0 pwl(0 0.7 0.9999us 0.7 1us 0 2us 0)
Mv1circuit3driverm0 out_v1circuit3driver v1circuit3 vdd vdd pmos_lvt nfin = 24
Mv1circuit3driverm1 out_v1circuit3driver v1circuit3 0 0 nmos_lvt nfin = 16

V2CIRCUIT3 v2circuit3 0 pwl(0 0.7 0.9999us 0.7 1us 0 2us 0)
Mv2circuit3driverm0 out_v2circuit3driver v2circuit3 vdd vdd pmos_lvt nfin = 24
Mv2circuit3driverm1 out_v2circuit3driver v2circuit3 0 0 nmos_lvt nfin = 16

Mgate1circuit3m0 out_gate1circuit3 out_v1circuit3driver gate1circuit3mid vdd pmos_lvt nfin = 12
Mgate1circuit3m1 gate1circuit3mid out_v2circuit3driver vdd vdd pmos_lvt nfin = 12
Mgate1circuit3m2 out_gate1circuit3 out_v1circuit3driver 0 0 nmos_lvt nfin = 4
Mgate1circuit3m3 out_gate1circuit3 out_v2circuit3driver 0 0 nmos_lvt nfin = 4

Mgate2circuit3m0 out_gate2circuit3 out_gate1circuit3 vdd vdd pmos_lvt nfin = 27
Mgate2circuit3m1 out_gate2circuit3 out